In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [7]:
len(twenty_train.data)

2257

In [9]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [10]:
count_vect = CountVectorizer(max_features=1000)

In [11]:
bow_transformer = count_vect.fit(twenty_train.data)

In [12]:
X_train_counts = bow_transformer.transform(twenty_train.data)
X_test_counts = bow_transformer.transform(twenty_test.data)

In [13]:
# Create classifiers
lr = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=100)

In [14]:
lr.fit(X_train_counts, twenty_train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
y_pred = lr.predict(X_test_counts)

In [17]:
y_pred[:20]

array([2, 2, 2, 0, 3, 0, 1, 3, 0, 1, 1, 3, 0, 3, 1, 2, 1, 3, 0, 0])

In [18]:
accuracy_score(twenty_test.target, y_pred)

0.8528628495339547

In [19]:
rfc.fit(X_train_counts, twenty_train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
y_pred = rfc.predict(X_test_counts)
accuracy_score(twenty_test.target, y_pred)

0.8275632490013316

In [27]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
epochs = 5

model = Sequential()
model.add(Dense(512, input_shape=(1000,)))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

y_train = keras.utils.to_categorical(twenty_train.target, 4)
y_test = keras.utils.to_categorical(twenty_test.target, 4)

history = model.fit(X_train_counts, y_train,
                    batch_size=1,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 2031 samples, validate on 226 samples
Epoch 1/5
2031/2031 [==============================] - 1s 579us/step - loss: 0.6102 - acc: 0.7878 - val_loss: 0.2351 - val_acc: 0.9292
Epoch 2/5
2031/2031 [==============================] - 1s 423us/step - loss: 0.1459 - acc: 0.9734 - val_loss: 0.3333 - val_acc: 0.8717
Epoch 3/5
2031/2031 [==============================] - 1s 451us/step - loss: 0.0888 - acc: 0.9828 - val_loss: 0.1327 - val_acc: 0.9646
Epoch 4/5
2031/2031 [==============================] - 1s 426us/step - loss: 0.0417 - acc: 0.9956 - val_loss: 0.1232 - val_acc: 0.9558
Epoch 5/5
2031/2031 [==============================] - 1s 421us/step - loss: 0.0275 - acc: 0.9980 - val_loss: 0.1085 - val_acc: 0.9690


In [28]:
y_pred = model.predict(X_test_counts)
accuracy_score(twenty_test.target, np.argmax(y_pred, axis=1))

0.8754993342210386

In [ ]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])


In [ ]:
x_train.shape